In [98]:
import pandas as pd

Data = pd.read_csv('HW1_census-tract_data.csv')
Data.loc[0,:]
Data

,Tract,Total poupulation(thousands),Median school years,Total employment(thousands),Health services employment(hundreds),"Median value home($10,000s)"
0,1,5.935,14.2,2.265,2.27,2.91
1,2,1.523,13.1,0.597,0.75,2.62
2,3,2.599,12.7,1.237,1.11,1.72
3,4,4.009,15.2,1.649,0.81,3.02
4,5,4.687,14.7,2.312,2.50,2.22
5,6,8.044,15.6,3.641,4.51,2.36
6,7,2.766,13.3,1.244,1.03,1.97
7,8,6.538,17.0,2.618,2.39,1.85
8,9,6.451,12.9,3.147,5.52,2.01
9,10,3.314,12.2,1.606,2.18,1.82


In [99]:
import numpy as np
#Column6_Multiplied_by_100 = Data['Median value home($10,000s)'].apply(lambda x:x*100)
Data.loc[:,'Median value home($10,000s)'] *= 100

# Drop "tract"
Drop_Tract = Data.drop(columns=['Tract'])
Column6_Multiplied_by_100 = Drop_Tract.to_numpy()
Column6_Multiplied_by_100
#np.array(Column6_Multiplied_by_100).shape
#Column6_Multiplied_by_100 = np.resize(Column6_Multiplied_by_100, [14,1])

array([[  5.935,  14.2  ,   2.265,   2.27 , 291.   ],
       [  1.523,  13.1  ,   0.597,   0.75 , 262.   ],
       [  2.599,  12.7  ,   1.237,   1.11 , 172.   ],
       [  4.009,  15.2  ,   1.649,   0.81 , 302.   ],
       [  4.687,  14.7  ,   2.312,   2.5  , 222.   ],
       [  8.044,  15.6  ,   3.641,   4.51 , 236.   ],
       [  2.766,  13.3  ,   1.244,   1.03 , 197.   ],
       [  6.538,  17.   ,   2.618,   2.39 , 185.   ],
       [  6.451,  12.9  ,   3.147,   5.52 , 201.   ],
       [  3.314,  12.2  ,   1.606,   2.18 , 182.   ],
       [  3.777,  13.   ,   2.119,   2.83 , 180.   ],
       [  1.53 ,  13.8  ,   0.798,   0.84 , 425.   ],
       [  2.768,  13.6  ,   1.336,   1.75 , 264.   ],
       [  6.585,  14.9  ,   2.763,   1.91 , 317.   ]])

In [100]:
Covariance_Matrix_S = np.cov(Column6_Multiplied_by_100)
print(np.array(Covariance_Matrix_S).shape)


(14, 14)


##### Test

In [ ]:
A = [1,2]
B = [3,4]
S = np.cov(A,B)
print(S)